In [2]:
import sys, os
sys.version, os.getcwd()

('3.6.5 |Anaconda, Inc.| (default, Apr 29 2018, 16:14:56) \n[GCC 7.2.0]',
 '/home/jared/galvanize/Projects/prouster')

# Torch

In [ ]:
torch.__version__

In [26]:
import pandas as pd

In [73]:
PREFIXES = ['WP','EU','CW','TT','RF']
def clean_source_data(directory):
    data = pd.read_table(directory, header=None)
    data['prefix'] = data[0].apply(lambda x: str(x).split(']')[0][1:].strip()).str.upper()
    data['prompt'] = data[0].apply(lambda x: str(x).split(']')[-1].strip())
    data['prompt'] = data['prompt'].apply(lambda x: str(x).strip('"').replace('``','"').replace("''",'"'))
    data.drop(columns=0, inplace=True)
    #undesired tags: (['IP','OT','PM','MP','PI','CC'])]
    data = data[data['prefix'].isin(PREFIXES)]
    data.reset_index(inplace=True, drop=True)
    data.dropna(inplace=True)
    return data
data = clean_source_data('data/writingPrompts/train.wp_source')

In [74]:
data.tail()

,prefix,prompt
249867,WP,"You wake up , extremely thirsty and dressed in..."
249868,WP,"After many years , you finally decide to open ..."
249869,WP,In a world where people can only be killed by ...
249870,WP,"Use a lyric from a song , or even the whole so..."
249871,CW,"Write your hero into a corner , and let me get..."


In [75]:
rf = data['prompt'][data['prefix']=='RF']

In [76]:
rf[1171]

'" 500 Writing Prompts " Name the top ten things on your bucket list .'

In [33]:
prompts_as_csv = []
for prefix in PREFIXES:
    prompts_as_csv.append(data['prompt'][data['prefix']==prefix])

In [36]:
for prefix, prompts in zip(PREFIXES,prompts_as_csv):
    prompts.to_csv('data/writingPrompts/'+prefix+'.csv',index=False)

for prefix, prompts in zip(PREFIXES,prompts_as_csv):
    prompts.to_csv('data/strings/'+prefix+'.txt',index=False)

In [80]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import unicodedata
import string

all_letters = string.ascii_letters + " .,;'-"
n_letters = len(all_letters) + 1 # Plus EOS marker

def findFiles(path): return glob.glob(path)

# Turn a Unicode string to plain ASCII, thanks to http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

# Build the category_lines dictionary, a list of lines per category
category_lines = {}
all_categories = []
for filename in findFiles('data/writingPrompts/RF.csv'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

if n_categories == 0:
    raise RuntimeError('Data not found. Make sure that you downloaded data '
        'from https://download.pytorch.org/tutorial/data.zip and extract it to '
        'the current directory.')

print('# categories:', n_categories, all_categories)
print(unicodeToAscii("O'Néàl"))

# categories: 1 ['RF']
O'Neal


In [81]:
category_lines

{'RF': ['The worst job interview anyone has ever had',
  'A boy and his grandfather in rural New England watch Sputnik fly overhead in the autumn of  .',
  "It sucks , but you both know it 's better to just cut ties .",
  "Write a soldier 's journal entry on his first day at war . Then write his last journal entry",
  'A small town citizen sees their first automobile',
  'Through the eyes of someone hopeless with grief .',
  "He does n't know why she died .",
  '  Writing Prompts  Name the top ten things on your bucket list .',
  "You did n't know it was going to be the last time you 'd ever speak to them .",
  'In Baltimore it is illegal to take a lion to the movies . Explain the incident that occurred that led to this law being formed .',
  'Uh oh , the manager just laid-off Fred , the only person holding the entire team together .',
  "You 're driving home at night , and you notice something moving in the backseat from the rear-view mirror ...",
  'All your life , people praised you

In [82]:
import torch
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size

        self.i2h = nn.Linear(n_categories + input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(n_categories + input_size + hidden_size, output_size)
        self.o2o = nn.Linear(hidden_size + output_size, output_size)
        self.dropout = nn.Dropout(0.1)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, category, input, hidden):
        input_combined = torch.cat((category, input, hidden), 1)
        hidden = self.i2h(input_combined)
        output = self.i2o(input_combined)
        output_combined = torch.cat((hidden, output), 1)
        output = self.o2o(output_combined)
        output = self.dropout(output)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

In [83]:
import random

# Random item from a list
def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

# Get a random category and random line from that category
def randomTrainingPair():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    return category, line

In [84]:
def categoryTensor(category):
    li = all_categories.index(category)
    tensor = torch.zeros(1, n_categories)
    tensor[0][li] = 1
    return tensor

# One-hot matrix of first to last letters (not including EOS) for input
def inputTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li in range(len(line)):
        letter = line[li]
        tensor[li][0][all_letters.find(letter)] = 1
    return tensor

# LongTensor of second letter to end (EOS) for target
def targetTensor(line):
    letter_indexes = [all_letters.find(line[li]) for li in range(1, len(line))]
    letter_indexes.append(n_letters - 1) # EOS
    return torch.LongTensor(letter_indexes)

In [85]:
def randomTrainingExample():
    category, line = randomTrainingPair()
    category_tensor = categoryTensor(category)
    input_line_tensor = inputTensor(line)
    target_line_tensor = targetTensor(line)
    return category_tensor, input_line_tensor, target_line_tensor

In [86]:
criterion = nn.NLLLoss()

learning_rate = 0.0005

def train(category_tensor, input_line_tensor, target_line_tensor):
    target_line_tensor.unsqueeze_(-1)
    hidden = rnn.initHidden()

    rnn.zero_grad()

    loss = 0

    for i in range(input_line_tensor.size(0)):
        output, hidden = rnn(category_tensor, input_line_tensor[i], hidden)
        l = criterion(output, target_line_tensor[i])
        loss += l

    loss.backward()

    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)

    return output, loss.item() / input_line_tensor.size(0)

In [87]:
import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [89]:
rnn = RNN(n_letters, 128, n_letters)

n_iters = 500
print_every = 1
plot_every = n_iters
all_losses = []
total_loss = 0 # Reset every plot_every iters

start = time.time()

for iter in range(1, n_iters + 1):
    output, loss = train(*randomTrainingExample())
    total_loss += loss

    if iter % print_every == 0:
        print('%s (%d %d%%) %.4f' % (timeSince(start), iter, iter / n_iters * 100, loss))

    if iter % plot_every == 0:
        all_losses.append(total_loss / plot_every)
        total_loss = 0

0m 0s (1 0%) 4.0704
0m 0s (2 0%) 4.0752
0m 0s (3 0%) 4.0611
0m 0s (4 0%) 4.0587
0m 0s (5 1%) 4.0468
0m 0s (6 1%) 4.0320
0m 0s (7 1%) 4.0064
0m 0s (8 1%) 4.0247
0m 0s (9 1%) 4.0016
0m 0s (10 2%) 3.9959
0m 1s (11 2%) 3.9823
0m 1s (12 2%) 3.9654
0m 1s (13 2%) 3.9769
0m 1s (14 2%) 3.9592
0m 1s (15 3%) 3.9390
0m 1s (16 3%) 3.9355
0m 1s (17 3%) 3.9266
0m 1s (18 3%) 3.9333
0m 1s (19 3%) 3.9248
0m 1s (20 4%) 3.8879
0m 1s (21 4%) 3.9015
0m 1s (22 4%) 3.8815
0m 1s (23 4%) 3.8530
0m 1s (24 4%) 3.8790
0m 2s (25 5%) 3.8263
0m 2s (26 5%) 3.7984
0m 2s (27 5%) 3.8069
0m 2s (28 5%) 3.7802
0m 2s (29 5%) 3.7632
0m 2s (30 6%) 3.6982
0m 2s (31 6%) 3.7031
0m 2s (32 6%) 3.7404
0m 2s (33 6%) 3.6634
0m 2s (34 6%) 3.5804
0m 2s (35 7%) 3.4784
0m 3s (36 7%) 3.4938
0m 3s (37 7%) 3.4961
0m 3s (38 7%) 3.6044
0m 3s (39 7%) 3.5621
0m 3s (40 8%) 3.4429
0m 3s (41 8%) 3.4735
0m 3s (42 8%) 3.3229
0m 3s (43 8%) 3.3915
0m 3s (44 8%) 3.3401
0m 3s (45 9%) 3.2773
0m 3s (46 9%) 3.3949
0m 4s (47 9%) 3.3710
0m 4s (48 9%) 3.7264
0

0m 32s (354 70%) 3.0827
0m 32s (355 71%) 3.0705
0m 32s (356 71%) 3.0944
0m 32s (357 71%) 3.1288
0m 32s (358 71%) 3.0126
0m 32s (359 71%) 3.3215
0m 33s (360 72%) 3.0438
0m 33s (361 72%) 2.9805
0m 33s (362 72%) 3.0308
0m 33s (363 72%) 3.1075
0m 33s (364 72%) 3.0723
0m 33s (365 73%) 3.1376
0m 33s (366 73%) 3.0806
0m 34s (367 73%) 3.0174
0m 34s (368 73%) 2.8687
0m 34s (369 73%) 3.0633
0m 34s (370 74%) 3.8472
0m 34s (371 74%) 3.1990
0m 34s (372 74%) 3.0827
0m 34s (373 74%) 2.9400
0m 34s (374 74%) 3.2784
0m 34s (375 75%) 3.0108
0m 34s (376 75%) 2.9923
0m 34s (377 75%) 3.2527
0m 35s (378 75%) 2.8936
0m 35s (379 75%) 3.0878
0m 35s (380 76%) 2.8772
0m 35s (381 76%) 3.5109
0m 35s (382 76%) 3.0459
0m 35s (383 76%) 3.1416
0m 35s (384 76%) 3.0834
0m 35s (385 77%) 3.1388
0m 35s (386 77%) 3.0182
0m 35s (387 77%) 3.1225
0m 35s (388 77%) 3.0737
0m 36s (389 77%) 3.0417
0m 36s (390 78%) 2.8964
0m 36s (391 78%) 2.9712
0m 36s (392 78%) 3.0084
0m 36s (393 78%) 3.1131
0m 36s (394 78%) 2.9654
0m 36s (395 79%)

In [90]:
max_length = 50

# Sample from a category and starting letter
def sample(category, start_letter='A'):
    with torch.no_grad():  # no need to track history in sampling
        category_tensor = categoryTensor(category)
        input = inputTensor(start_letter)
        hidden = rnn.initHidden()

        output_name = start_letter

        for i in range(max_length):
            output, hidden = rnn(category_tensor, input[0], hidden)
            topv, topi = output.topk(random.randint(1,1))
            topi = topi[0][0]
            if topi == n_letters - 1:
                break
            else:
                letter = all_letters[topi]
                output_name += letter
            input = inputTensor(letter)

        return output_name

# Get multiple samples from one category and multiple starting letters
def samples(category, start_letters='ABC'):
    for start_letter in start_letters:
        print(sample(category, start_letter))

# samples('WP', 'RUSE')

# samples('EU', 'GERM')

# samples('CW', 'SPAN')

# samples('TT', 'CHIN')

samples('RF', 'IRLD')


I  a  a a  a a a a a a an a an a a a a a a te a a a
R  a  a a a a a a a a a a an a  a a a a a a a a a a
L e  a a a a a an a an a  a a  a a a a a a a a a a 
D  a a a  n a a  a a a a an an a  an a  a a a a an 


Rakov
Uanon
Shavan
Eakan
Garter
Eerter
Roun
Merter
Sanala
Parer
Alara
Nara
Cha
Houn
Iun
Nan
Iana
Rongan
Langan
Dangara
Santer
Chang
Oringel
Trang


# Keras Char-Level

In [2]:
import numpy as np
import pandas as pd

In [3]:
from keras.models import Sequential, Model, Input
from keras.layers import LSTM, TimeDistributed, Activation, Dense, Lambda

/home/jared/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
with open('data/writingPrompts/RF.txt') as f:
    lines = f.read().splitlines()

In [5]:
documents = ''.join(lines)

In [6]:
chars = list(set(documents))
VOCAB_SIZE = len(chars)

In [7]:
ix_to_char = {ix:char for ix, char in enumerate(chars)}
char_to_ix = {char:ix for ix, char in enumerate(chars)}


In [8]:
D_LENGTH = int(len(documents))
SEQ_LENGTH = 200

In [9]:
HIDDEN_DIM = 300
LAYER_NUM = 4
BATCH_SIZE = 100
temp = 3

In [10]:
X = np.zeros((len(documents)//SEQ_LENGTH, SEQ_LENGTH, VOCAB_SIZE))
y = np.zeros((len(documents)//SEQ_LENGTH, SEQ_LENGTH, VOCAB_SIZE))
for i in range(0, len(documents)//SEQ_LENGTH):
    X_sequence = documents[i*SEQ_LENGTH:(i+1)*SEQ_LENGTH]
    X_sequence_ix = [char_to_ix[value] for value in X_sequence]
    input_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))
    for j in range(SEQ_LENGTH):
        input_sequence[j][X_sequence_ix[j]] = 1.
    X[i] = input_sequence

    y_sequence = documents[i*SEQ_LENGTH+1:(i+1)*SEQ_LENGTH+1]
    y_sequence_ix = [char_to_ix[value] for value in y_sequence]
    target_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))
    for j in range(SEQ_LENGTH):
        #print(j)
        target_sequence[j][y_sequence_ix[j]] = 1.
    y[i] = target_sequence

In [11]:
model = Sequential()
model.add(LSTM(HIDDEN_DIM, input_shape=(None, VOCAB_SIZE), return_sequences=True))
for i in range(LAYER_NUM - 1):
    model.add(LSTM(HIDDEN_DIM, return_sequences=True))
model.add(TimeDistributed(Dense(VOCAB_SIZE)))
model.add(Lambda(lambda x: x / temp))
model.add(Activation('softmax'))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [12]:
def generate_text(model, length):
    ix = [np.random.randint(VOCAB_SIZE)]
    y_char = [ix_to_char[ix[-1]]]
    X = np.zeros((1, length, VOCAB_SIZE))
    for i in range(length):
        X[0, i, :][ix[-1]] = 1
        print(ix_to_char[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(ix_to_char[ix[-1]])
    return ('').join(y_char)

In [ ]:
nb_epoch = 0

print('\n\n')
model.fit(X, y, batch_size=BATCH_SIZE, verbose=1, nb_epoch=25)
nb_epoch += 1
generate_text(model, SEQ_LENGTH)
if nb_epoch % 2 == 0:
    model.save_weights('checkpoint_{}_epoch_{}.hdf5'.format(HIDDEN_DIM, nb_epoch))

/home/jared/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  after removing the cwd from sys.path.


Epoch 1/25


# Keras Word Level

In [16]:
republic = 'data/strings/RFs.txt'

In [17]:
import string

# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

# turn a doc into clean tokens
def clean_doc(doc):
    # replace '--' with a space ' '
    doc = doc.replace('--', ' ')
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # make lower case
    tokens = [word.lower() for word in tokens]
    return tokens

# save tokens to file, one dialog per line
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

# load document
in_filename = republic
doc = load_doc(in_filename)
print(doc[:200])

# clean document
tokens = clean_doc(doc)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

# organize into sequences of tokens
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
    # select sequence of tokens
    seq = tokens[i-length:i]
    # convert into a line
    line = ' '.join(seq)
    # store
    sequences.append(line)
print('Total Sequences: %d' % len(sequences))

# save sequences to file
out_filename = 'data/strings/RFsequences.txt'

The worst job interview anyone has ever had  A boy and his grandfather in rural New England watch Sputnik fly overhead in the autumn of 1957 .  It sucks , but you both know it 's better to just cut ti
['the', 'worst', 'job', 'interview', 'anyone', 'has', 'ever', 'had', 'a', 'boy', 'and', 'his', 'grandfather', 'in', 'rural', 'new', 'england', 'watch', 'sputnik', 'fly', 'overhead', 'in', 'the', 'autumn', 'of', 'it', 'sucks', 'but', 'you', 'both', 'know', 'it', 's', 'better', 'to', 'just', 'cut', 'ties', 'write', 'a', 'soldier', 's', 'journal', 'entry', 'on', 'his', 'first', 'day', 'at', 'war', 'then', 'write', 'his', 'last', 'journal', 'entry', 'a', 'small', 'town', 'citizen', 'sees', 'their', 'first', 'automobile', 'through', 'the', 'eyes', 'of', 'someone', 'hopeless', 'with', 'grief', 'he', 'does', 'nt', 'know', 'why', 'she', 'died', 'writing', 'prompts', 'name', 'the', 'top', 'ten', 'things', 'on', 'your', 'bucket', 'list', 'you', 'did', 'nt', 'know', 'it', 'was', 'going', 'to', 'be',

In [18]:
save_doc(sequences, out_filename)

In [19]:
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

In [20]:
!# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text
 
# load
in_filename = 'data/strings/republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
 
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1
 
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]

In [21]:
y

array([  35, 3887, 2035, ...,  320,   85,  314])

In [22]:
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

MemoryError: 

In [9]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=5)
 
# save the model to file
model.save('model.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

MemoryError: 

In [ ]:
keras.utils.plot_model(model,'model.png')